In [98]:
import pandas as pd
import numpy as np
import networkx as nx

# 1. Load CSV
df = pd.read_csv("../datasets/GoldCoast_network.csv")

# 2. Compute total capacity = capacity per lane × number of lanes
df['TotalCapacity'] = df['Capacity'] * df['Lanes']

# 3. Create directed graph
G = nx.DiGraph()

# 4. Add edges with attributes (including total capacity)
for _, row in df.iterrows():
    G.add_edge(
        row['From'],
        row['To'],
        TotalCapacity=row['TotalCapacity'],
        Length=row['Length']
    )

# 5. Example: print an edge with its attributes
u, v = list(G.edges())[1]
print("Example edge:", u, "→", v)
print(G[u][v])


Example edge: 1371.0 → 1333.0
{'TotalCapacity': np.float64(1800.0), 'Length': np.float64(0.49)}


### Do alternative way

In [99]:
def directed_to_undirected(G):
    """
    G: nx.DiGraph with edge attributes:
       - TotalCapacity
       - Length

    Returns:
        H: nx.Graph with merged edges
    """
    H = nx.Graph()

    for u, v, data in G.edges(data=True):
        cap = data["TotalCapacity"]

        if H.has_edge(u, v):
            H[u][v]["Capacity"] += cap
        else:
            H.add_edge(
                u, v,
                Capacity=cap,
            )

    return H


In [100]:
H = directed_to_undirected(G)

In [101]:
u, v = list(H.edges())[0]
print("Example edge:", u, "→", v)
print(H[u][v])

Example edge: 1.0 → 1371.0
{'Capacity': np.float64(3600.0)}


In [102]:
def edge_to_node_graph(H):

    # Remember that the node names in a line graph is the edge it represents, so an egde node might be called (1, 10) meaning the node that represents the edge going from node 1 to 10
    L = nx.line_graph(H)

    # Copy edge attributes → node attributes
    # Checks if (u,v) node is L if not then that node must be called (v,u) instead
    for u, v, data in H.edges(data=True):
        edge_node = (u, v) if (u, v) in L else (v, u)

        L.nodes[edge_node]["Capacity"] = data["Capacity"]
    return L


In [103]:
L = edge_to_node_graph(H)

In [104]:
print("Edge-nodes (roads):")
for n, d in L.nodes(data=True):
    print(n, d)

print("\nConnections (shared intersections):")
print(list(L.edges()))


Edge-nodes (roads):
(np.float64(1.0), np.float64(1371.0)) {'Capacity': np.float64(3600.0)}
(np.float64(3.0), np.float64(2402.0)) {'Capacity': np.float64(4400.0)}
(np.float64(4.0), np.float64(1875.0)) {'Capacity': np.float64(1600.0)}
(np.float64(5.0), np.float64(1880.0)) {'Capacity': np.float64(1600.0)}
(np.float64(6.0), np.float64(1894.0)) {'Capacity': np.float64(1600.0)}
(np.float64(7.0), np.float64(1970.0)) {'Capacity': np.float64(1600.0)}
(np.float64(8.0), np.float64(1977.0)) {'Capacity': np.float64(1200.0)}
(np.float64(9.0), np.float64(3448.0)) {'Capacity': np.float64(1200.0)}
(np.float64(11.0), np.float64(2867.0)) {'Capacity': np.float64(6000.0)}
(np.float64(12.0), np.float64(3627.0)) {'Capacity': np.float64(800.0)}
(np.float64(3627.0), np.float64(188.0)) {'Capacity': np.float64(800.0)}
(np.float64(15.0), np.float64(3441.0)) {'Capacity': np.float64(2800.0)}
(np.float64(16.0), np.float64(3576.0)) {'Capacity': np.float64(1600.0)}
(np.float64(17.0), np.float64(3839.0)) {'Capacity': n

### Initial loads

In [107]:
# Alpha is the value that determines the percentage load on the roads
def initialize_loads(L, alpha=0.8):

    for n, d in L.nodes(data=True):
        d["load"] = alpha * d["Capacity"]
        d["failed"] = False


### Function for one Cascading step

In [108]:
def cascade_step(L):
    newly_failed = []

    # Identify overloaded nodes
    for n, d in L.nodes(data=True):
        if not d["failed"] and d["load"] > d["Capacity"]:
            newly_failed.append(n)

    if not newly_failed:
        return False

    # Process failures
    for n in newly_failed:
        d = L.nodes[n]
        d["failed"] = True

        neighbors = [v for v in L.neighbors(n) if not L.nodes[v]["failed"]]
        if neighbors:
            redistributed = d["load"] / len(neighbors)
            for v in neighbors:
                L.nodes[v]["load"] += redistributed

        d["load"] = 0.0  # failed road carries no traffic

    return True


### Running of cascade effekt

In [109]:
def run_cascade(L, max_iter=1000):
    for _ in range(max_iter):
        if not cascade_step(L):
            break


In [110]:


initialize_loads(L, alpha=0.8)

# Optional: force initial failure
info = {}

for node in range(len(list(L.nodes()))):
    L_run = L
    L_run.nodes[list(L_run.nodes)[node]]["load"] *= 2

    run_cascade(L_run)

    failed_nodes = [n for n, d in L_run.nodes(data=True) if d["failed"]]
    print(f"Failed roads: {len(failed_nodes)}")
    info[node] = failed_nodes


Failed roads: 5374
Failed roads: 5374
Failed roads: 5374
Failed roads: 5374
Failed roads: 5374
Failed roads: 5374
Failed roads: 5374
Failed roads: 5374
Failed roads: 5375
Failed roads: 5375
Failed roads: 5375
Failed roads: 5375
Failed roads: 5375
Failed roads: 5375
Failed roads: 5377
Failed roads: 5377
Failed roads: 5377
Failed roads: 5377
Failed roads: 5378
Failed roads: 5379
Failed roads: 5380
Failed roads: 5381
Failed roads: 5381
Failed roads: 5381
Failed roads: 5382
Failed roads: 5383
Failed roads: 5383
Failed roads: 5384
Failed roads: 5385
Failed roads: 5386
Failed roads: 5387
Failed roads: 5388
Failed roads: 5389
Failed roads: 5390
Failed roads: 5390
Failed roads: 5390
Failed roads: 5391
Failed roads: 5392
Failed roads: 5392
Failed roads: 5393
Failed roads: 5393
Failed roads: 5393
Failed roads: 5393
Failed roads: 5394
Failed roads: 5395
Failed roads: 5396
Failed roads: 5397
Failed roads: 5398
Failed roads: 5400
Failed roads: 5400
Failed roads: 5401
Failed roads: 5401
Failed roads

In [111]:
info[0]

[(np.float64(1.0), np.float64(1371.0)),
 (np.float64(3.0), np.float64(2402.0)),
 (np.float64(4.0), np.float64(1875.0)),
 (np.float64(5.0), np.float64(1880.0)),
 (np.float64(6.0), np.float64(1894.0)),
 (np.float64(7.0), np.float64(1970.0)),
 (np.float64(8.0), np.float64(1977.0)),
 (np.float64(9.0), np.float64(3448.0)),
 (np.float64(12.0), np.float64(3627.0)),
 (np.float64(3627.0), np.float64(188.0)),
 (np.float64(15.0), np.float64(3441.0)),
 (np.float64(16.0), np.float64(3576.0)),
 (np.float64(17.0), np.float64(3839.0)),
 (np.float64(19.0), np.float64(2795.0)),
 (np.float64(20.0), np.float64(2166.0)),
 (np.float64(26.0), np.float64(4769.0)),
 (np.float64(27.0), np.float64(3885.0)),
 (np.float64(30.0), np.float64(3730.0)),
 (np.float64(38.0), np.float64(3174.0)),
 (np.float64(39.0), np.float64(2852.0)),
 (np.float64(42.0), np.float64(3202.0)),
 (np.float64(44.0), np.float64(2531.0)),
 (np.float64(45.0), np.float64(2816.0)),
 (np.float64(1978.0), np.float64(46.0)),
 (np.float64(54.0), np.

In [112]:
initialize_loads(L, alpha=0.8)

# Optional: force initial failure
info = {}

for node in range(len(list(L.nodes()))):
    L_run = L
    L_run.nodes[list(L_run.nodes)[node]]["failure"] = True

    run_cascade(L_run)

    failed_nodes = [n for n, d in L_run.nodes(data=True) if d["failed"]]
    print(f"Failed roads: {len(failed_nodes)}")
    info[node] = failed_nodes

Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed roads: 0
Failed r